In [ ]:
import numpy as np
import cv2  # For image processing
from scipy.fft import fft
from scipy.signal import welch
from scipy import signal
from scipy.ndimage import gaussian_filter
from skimage.color import rgb2gray

# Constants
IMG_SIZE = 1024  # Max image size
FREQ_BANDS = {"essential": (4, 8), "parkinsonian": (3, 7)}  # Hz ranges for tremors

# Load Image
def load_image(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img

# Convert pixel (i,j) to Cartesian (x,y)
def pixel_to_cartesian(img, i, j):
    x0, y0 = 0, IMG_SIZE  # center coordinates if starting at max r
    x1, y1 = IMG_SIZE, 0
    x = x0 + j * (x1 - x0) / (img.shape[1] - 1)
    y = y1 + i * (y0 - y1) / (img.shape[0] - 1)
    return x, y

# Convert (x,y) to polar coordinates (r, theta)
def cartesian_to_polar(x, y):
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    return r, theta

# Extract the spiral data in polar coordinates
def extract_spiral_data(img):
    r_vals, theta_vals = [], []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            x, y = pixel_to_cartesian(img, i, j)
            r, theta = cartesian_to_polar(x, y)
            r_vals.append(r)
            theta_vals.append(theta)
    r_vals, theta_vals = np.array(r_vals), np.array(theta_vals)
    return r_vals, theta_vals

# Frequency Domain Analysis using FFT
def fft_analysis(r_vals, theta_vals, freq_band):
    fft_result = fft(r_vals)
    freqs = np.fft.fftfreq(len(r_vals))
    # Filter for specific tremor frequency band
    freq_indices = np.where((freqs >= freq_band[0]) & (freqs <= freq_band[1]))
    tremor_fft = fft_result[freq_indices]
    peak_freq = freqs[np.argmax(np.abs(tremor_fft))]
    peak_amplitude = np.max(np.abs(tremor_fft))
    return peak_freq, peak_amplitude

# Power Spectral Density (PSD)
def psd_analysis(r_vals, fs=100):  # Assuming 100Hz sampling rate
    f, Pxx = welch(r_vals, fs=fs)
    return f, Pxx

# Wavelet Transform for Time-Frequency Analysis
def wavelet_transform(r_vals, widths=np.arange(1, 31)):
    cwt_result = signal.cwt(r_vals, signal.morlet, widths)
    return cwt_result

# Geometric Features
def geometric_features(r_vals, theta_vals):
    zero_crossing_rate = np.sum(np.diff(np.sign(r_vals)) != 0) / len(r_vals)
    curvature = np.abs(np.gradient(np.gradient(r_vals)))  # Approximate curvature
    roundness = np.var(r_vals)  # Variation in radius
    symmetry = np.mean(r_vals[:len(r_vals)//2]) - np.mean(r_vals[len(r_vals)//2:])
    return zero_crossing_rate, curvature, roundness, symmetry

# Main function
def analyze_tremor(filepath):
    img = load_image(filepath)
    r_vals, theta_vals = extract_spiral_data(img)
    
    # Frequency Domain Features
    essential_freq, essential_amp = fft_analysis(r_vals, theta_vals, FREQ_BANDS["essential"])
    parkinsonian_freq, parkinsonian_amp = fft_analysis(r_vals, theta_vals, FREQ_BANDS["parkinsonian"])
    
    # Power Spectral Density
    freqs, psd_vals = psd_analysis(r_vals)
    
    # Time-Frequency Analysis using CWT
    cwt_result = wavelet_transform(r_vals)
    
    # Geometric Features
    zero_crossing_rate, curvature, roundness, symmetry = geometric_features(r_vals, theta_vals)
    
    return {
        "essential_freq": essential_freq,
        "essential_amp": essential_amp,
        "parkinsonian_freq": parkinsonian_freq,
        "parkinsonian_amp": parkinsonian_amp,
        "psd": (freqs, psd_vals),
        "cwt": cwt_result,
        "zero_crossing_rate": zero_crossing_rate,
        "curvature": curvature,
        "roundness": roundness,
        "symmetry": symmetry
    }

# Example usage
filepath = "path/to/patient_spiral_image.png"
results = analyze_tremor(filepath)
print(results)
